## Bayesian Theorem
Bayesian Theorem describes the probability of an event, based on prior knowledge of conditions that might be related to the event. It is mathematically expressed as:
$$
P(A|B) = \frac{P(B|A) * P(A)}{P(B)}
$$

Where:
- **P(A|B)** is the posterior probability: the probability of hypothesis A given the data B.
- **P(B|A)** is the likelihood: the probability of data B given that hypothesis A is true.
- **P(A)** is the prior probability: the initial probability of hypothesis A.
- **P(B)** is the marginal likelihood: the total probability of data B under all hypotheses.

Bayesian methods are widely used in various fields, including statistics, machine learning, and artificial intelligence, for tasks such as classification, regression, and decision-making under uncertainty.

### 1. Disease Diagnosis Example

In [1]:
# Symptoms: fever, cough, fatigue
diseases = {
    'flu': {
        'prior': 0.05,      # 5% prevalence
        'symptoms': {
            'fever': 0.8,
            'cough': 0.9,
            'fatigue': 0.7
        }
    },
    'cold': {
        'prior': 0.15,      # 15% prevalence  
        'symptoms': {
            'fever': 0.1,
            'cough': 0.8,
            'fatigue': 0.6
        }
    },
    'allergies': {
        'prior': 0.10,      # 10% prevalence
        'symptoms': {
            'fever': 0.01,
            'cough': 0.7,
            'fatigue': 0.3
        }
    },
    'healthy': {
        'prior': 0.70,      # 70% healthy
        'symptoms': {
            'fever': 0.02,
            'cough': 0.1,
            'fatigue': 0.2
        }
    }
}

In [2]:
def diagnose(symptoms_present, diseases):
    """Diagnose based on symptoms using Bayes' Theorem"""
    results = {}
    total_prob = 0
    
    # Calculate unnormalized probabilities
    for disease, info in diseases.items():
        prob = info['prior']
        for symptom, present in symptoms_present.items():
            if present:
                prob *= info['symptoms'][symptom]
            else:
                prob *= (1 - info['symptoms'][symptom])
        results[disease] = prob
        total_prob += prob
    
    # Normalize
    if total_prob > 0:
        for disease in results:
            results[disease] /= total_prob
    
    return results


In [3]:
# Test cases
test_cases = [
    {'fever': True, 'cough': True, 'fatigue': True},      # Likely flu
    {'fever': False, 'cough': True, 'fatigue': True},     # Could be cold/allergies
    {'fever': False, 'cough': False, 'fatigue': False},   # Likely healthy
]

print("Medical Diagnosis with Bayes' Theorem")
print("=" * 50)

for i, symptoms in enumerate(test_cases, 1):
    diagnosis = diagnose(symptoms, diseases)
    print(f"\nCase {i}: Symptoms = {symptoms}")
    
    # Sort by probability
    sorted_diagnosis = sorted(diagnosis.items(), key=lambda x: x[1], reverse=True)
    for disease, prob in sorted_diagnosis:
        print(f"  {disease:12}: {prob:.4f} ({prob:.1%})")
    
    top_diagnosis = sorted_diagnosis[0][0]
    print(f"  Most likely: {top_diagnosis.upper()}")

Medical Diagnosis with Bayes' Theorem

Case 1: Symptoms = {'fever': True, 'cough': True, 'fatigue': True}
  flu         : 0.7662 (76.6%)
  cold        : 0.2189 (21.9%)
  healthy     : 0.0085 (0.9%)
  allergies   : 0.0064 (0.6%)
  Most likely: FLU

Case 2: Symptoms = {'fever': False, 'cough': True, 'fatigue': True}
  cold        : 0.6136 (61.4%)
  allergies   : 0.1969 (19.7%)
  healthy     : 0.1299 (13.0%)
  flu         : 0.0597 (6.0%)
  Most likely: COLD

Case 3: Symptoms = {'fever': False, 'cough': False, 'fatigue': False}
  healthy     : 0.9394 (93.9%)
  allergies   : 0.0395 (4.0%)
  cold        : 0.0205 (2.1%)
  flu         : 0.0006 (0.1%)
  Most likely: HEALTHY


### 2. A/B Testing Example

In [4]:
import numpy as np

def bayesian_analysis(successes_A, trials_A, successes_B, trials_B, prior_alpha=1, prior_beta=1):
    """
    Bayesian A/B test analysis using Beta-Binomial conjugate prior
    """
    from scipy.stats import beta
    
    # Posterior distributions (Beta distribution)
    # Using uniform prior Beta(1,1)
    posterior_A = beta(prior_alpha + successes_A, prior_beta + (trials_A - successes_A))
    posterior_B = beta(prior_alpha + successes_B, prior_beta + (trials_B - successes_B))
    
    # Probability that B is better than A
    samples = 100000
    samples_A = posterior_A.rvs(samples)
    samples_B = posterior_B.rvs(samples)
    prob_B_better = np.mean(samples_B > samples_A)
    
    return {
        'posterior_A': posterior_A,
        'posterior_B': posterior_B,
        'prob_B_better': prob_B_better,
        'expected_A': successes_A / trials_A,
        'expected_B': successes_B / trials_B
    }


In [5]:
# Simulated A/B test results
# Version A: 45 conversions out of 1000
# Version B: 55 conversions out of 1000
results = bayesian_analysis(45, 1000, 55, 1000)

print("Bayesian A/B Testing Analysis")
print("=" * 40)
print(f"Version A: {45}/1000 = {45/1000:.3f}")
print(f"Version B: {55}/1000 = {55/1000:.3f}")
print(f"Probability that B is better than A: {results['prob_B_better']:.4f}")
print(f"Expected improvement: {(55/1000 - 45/1000)/ (45/1000):.1%}")

# Decision making
if results['prob_B_better'] > 0.95:
    print("✅ STRONG evidence for Version B")
elif results['prob_B_better'] > 0.90:
    print("✅ MODERATE evidence for Version B") 
elif results['prob_B_better'] > 0.80:
    print("✅ WEAK evidence for Version B")
else:
    print("❌ Insufficient evidence to choose Version B")

Bayesian A/B Testing Analysis
Version A: 45/1000 = 0.045
Version B: 55/1000 = 0.055
Probability that B is better than A: 0.8458
Expected improvement: 22.2%
✅ WEAK evidence for Version B
